In [1]:
using CausalELM
using CSV
using DataFrames
using Random

In [2]:
pension_df = CSV.read("pension.csv", DataFrame)
pension_df = pension_df[:, [10, 22, 13, 14, 15, 18, 20, 17, 24, 33]]
covariates, treatment, outcome = pension_df[:, 3:end], pension_df[:, 2], pension_df[:, 1]

(9915×8 DataFrame
  Row │ age    inc    fsize  marr   twoearn  db     pira   hown  
      │ Int64  Int64  Int64  Int64  Int64    Int64  Int64  Int64 
──────┼──────────────────────────────────────────────────────────
    1 │    31  28146      5      1        0      0      0      1
    2 │    52  32634      5      0        0      0      0      1
    3 │    50  52206      3      1        1      0      1      1
    4 │    28  45252      4      1        1      0      0      0
    5 │    42  33126      3      0        0      1      0      1
    6 │    49  76860      6      1        1      1      0      1
    7 │    40  57477      4      1        1      1      0      1
    8 │    58  14637      1      0        0      0      0      0
  ⋮   │   ⋮      ⋮      ⋮      ⋮       ⋮       ⋮      ⋮      ⋮
 9909 │    28  31926      2      1        1      0      0      0
 9910 │    49  64215      4      1        1      0      1      1
 9911 │    34  13500      1      0        0      1      0      0
 9912 

In [3]:
covariates.inc = (covariates.inc .- minimum(covariates.inc))/(maximum(covariates.inc) - minimum(covariates.inc))
covariates.age = (covariates.age .- minimum(covariates.age))/(maximum(covariates.age) - minimum(covariates.age))
covariates.fsize = (covariates.fsize .- minimum(covariates.fsize))/(maximum(covariates.fsize) - minimum(covariates.fsize))
covariates.marr = (covariates.marr .- minimum(covariates.marr))/(maximum(covariates.marr) - minimum(covariates.marr))
covariates

Row,age,inc,fsize,marr,twoearn,db,pira,hown
,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64
1,0.153846,0.125821,0.333333,1.0,0,0,0,1
2,0.692308,0.144156,0.333333,0.0,0,0,0,1
3,0.641026,0.224115,0.166667,1.0,1,0,1,1
4,0.0769231,0.195705,0.25,1.0,1,0,0,0
5,0.435897,0.146166,0.166667,0.0,0,1,0,1
6,0.615385,0.324836,0.416667,1.0,1,1,0,1
7,0.384615,0.245649,0.25,1.0,1,1,0,1
8,0.846154,0.0706319,0.0,0.0,0,0,0,0
9,0.102564,0.0376875,0.25,0.0,0,0,0,0


In [59]:
dml = DoubleMachineLearning(covariates, treatment, outcome, activation=swish)

DoubleMachineLearning([0.46153846153846156 0.21734974017060496 … 0.0 1.0; 0.5897435897435898 0.05495636827139916 … 0.0 0.0; … ; 0.02564102564102564 0.11648200804000393 … 0.0 1.0; 0.6410256410256411 0.22411510932444356 … 1.0 1.0], [0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0  …  0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0], [18800.0, 500.0, 5600.0, 62535.0, -5100.0, 9145.0, 25999.0, 0.0, 2150.0, 5000.0  …  189000.0, 14400.0, 240.0, 249.0, -928.0, 107750.0, 0.0, 114335.0, 10500.0, 8849.0], "ATE", false, "regression", CausalELM.swish, 9915, 100, 6, 24, NaN, 5)

In [60]:
r_learner = RLearner(covariates, treatment, outcome, activation=swish)

RLearner([0.1282051282051282 0.11108932248259633 … 0.0 1.0; 0.6923076923076923 0.1186881066771252 … 0.0 1.0; … ; 0.20512820512820512 0.07500735366212374 … 0.0 1.0; 0.41025641025641024 0.11607755662319835 … 0.0 1.0], [0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0  …  0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-8900.0, -4800.0, 27500.0, -1650.0, -2000.0, 30740.0, 2859.0, -2150.0, 0.0, 11599.0  …  43599.0, -7200.0, 23309.0, 8774.0, 6500.0, -400.0, 22700.0, 7399.0, -5400.0, 1499.0], "CATE", false, "regression", CausalELM.swish, 9915, 100, 6, 32, [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN], 5)

In [61]:
dre = DoublyRobustLearner(covariates, treatment, outcome, activation=swish)

DoublyRobustLearner([0.6410256410256411 0.1558486126090793 … 0.0 1.0; 0.23076923076923078 0.06633003235611334 … 0.0 0.0; … ; 0.6153846153846154 0.06843808216491813 … 0.0 0.0; 0.7435897435897436 0.2292994411216786 … 0.0 1.0], [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0  …  0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0], [100.0, 0.0, 14350.0, 4600.0, 84248.0, -1800.0, 1020.0, 2280.0, 14699.0, 881.0  …  367.0, -5600.0, -5400.0, 5674.0, 12211.0, 32500.0, 1152.0, 2182.0, 0.0, 330.0], "CATE", false, "regression", CausalELM.swish, 9915, 100, 6, 32, [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN], 2)

In [62]:
estimate_causal_effect!(dml)

8804.269472283213

In [63]:
estimate_causal_effect!(r_learner)

9915-element Vector{Float64}:
  1033.275328379404
  3897.6188907530145
 27094.516749605616
  8327.283149032586
  6781.702531736929
 50200.72898282418
   618.590315821573
  6647.26749174192
  4325.783318029439
 16617.629336705013
     ⋮
 25103.616301146572
 40417.24987461999
  6976.012498684692
  8869.662932387795
 -1030.3323016387612
  4912.327776140574
  2840.9932292653525
  3323.126233753097
 21356.54170795394

In [64]:
estimate_causal_effect!(dre)

9915-element Vector{Float64}:
  8651.259686332123
  2763.6426805062965
  4281.08620983512
  6996.106017505121
 37295.1224689869
  3425.2628336886887
  7259.653364085303
  3931.840707261489
  3390.6489181977217
   396.19186564028234
     ⋮
 13778.740930336877
 13824.272936865971
   770.8718719469387
  5661.227928432385
 10218.778717409776
  3707.70741363045
  2089.690748271022
  3767.843767168565
 17841.535784697724

In [9]:
summarize(dml)

Dict{Any, Any} with 11 entries:
  "Activation Function"    => relu
  "Quantity of Interest"   => "ATE"
  "Sample Size"            => 9915
  "Number of Machines"     => 100
  "Causal Effect"          => 8823.5
  "Number of Neurons"      => 24
  "Task"                   => "regression"
  "Time Series/Panel Data" => false
  "Standard Error"         => NaN
  "p-value"                => NaN
  "Number of Features"     => 6

In [10]:
summarize(r_learner)

Dict{Any, Any} with 11 entries:
  "Activation Function"    => relu
  "Quantity of Interest"   => "CATE"
  "Sample Size"            => 9915
  "Number of Machines"     => 100
  "Causal Effect"          => [4085.58, 15773.5, 38901.8, 3825.38, 11964.8, 267…
  "Number of Neurons"      => 32
  "Task"                   => "regression"
  "Time Series/Panel Data" => false
  "Standard Error"         => NaN
  "p-value"                => NaN
  "Number of Features"     => 6

In [12]:
validate(dml)

(Dict("0.1 Standard Deviations from Observed Outcomes" => -8079.331571957283, "0.075 Standard Deviations from Observed Outcomes" => -6089.203934396697, "0.025 Standard Deviations from Observed Outcomes" => -7522.457852582857, "0.05 Standard Deviations from Observed Outcomes" => -12933.100480526482), 2.6894381997142496, Matrix{Float64}(undef, 0, 9))

In [13]:
validate(r_learner)

(Dict("0.1 Standard Deviations from Observed Outcomes" => 155340.94980401796, "0.075 Standard Deviations from Observed Outcomes" => 559571.3301919985, "0.025 Standard Deviations from Observed Outcomes" => 274961.5431470514, "0.05 Standard Deviations from Observed Outcomes" => 345062.1310616215), 2.8689322412325833, Matrix{Float64}(undef, 0, 9))